In [1]:
#import pya declarations
import sys
import klayout.db as pya
from pya import Box, Point, Polygon, Text, Trans, LayerInfo, \
    PCellDeclarationHelper, DPoint, DPath, Path, ShapeProcessor, \
    Library, CellInstArray

#register and load siepic_tools
sys.path.append("C://users/root/Documents/GitHub/SiEPIC-Tools/klayout_dot_config/python/")
import siepic_tools

#load tech layout file
from zeropdk import Tech

#technology data folder
tech_path = r"C:\Users\root\Documents\GitHub\SiEPIC-Tools\klayout_dot_config\tech\GSiP"
lib = Tech.load_from_xml(tech_path+r"\klayout_Layers_GSiP.lyp")

#import library pcells
sys.path.append(tech_path+r"\pymacros")
from GSiP_Library import *
GSiP()

from SiEPIC.utils import arc_xy

print("done importing libraries")

C:\Users\root\Documents\GitHub\SiEPIC-Tools\klayout_dot_config\tech\GSiP\klayout_Layers_GSiP.lyp
Initializing 'GSiP' Library.
done importing libraries


In [2]:
#create layout
layout = pya.Layout()
dbu = layout.dbu = 0.001
TOP = layout.create_cell("TOP")

In [4]:
# Define Layer mapping and floor plan
LayerSiN = layout.layer(lib.layers['Si'])
fpLayerN = TOP.layout().layer(lib.layers['FloorPlan'])
TextLayerN = TOP.layout().layer(lib.layers['Text'])

# Draw the floor plan
TOP.shapes(fpLayerN).insert(Box(-610/dbu,-405/dbu, 610/dbu, 405/dbu))

In [5]:
layout.prune_subcells(TOP.cell_index(), 1000)

In [6]:
# Import Grating couplers
GC_imported = layout.create_cell("Grating_Coupler_13deg_TE_1550_Oxide", "GSiP").cell_index()

gc_length = 41
GC_pitch = 127

t = Trans(Trans.R0, 0, 0)
TOP.insert(CellInstArray(GC_imported, t, DPoint(0,GC_pitch).to_itype(dbu), Point(0,0), 2, 1))


In [7]:
#draw waveguide connecting grating couplers
path_pts = [Point(each[0], each[1]) for each in [[0,0], [300,0], [300, 127], [0,127]]]
path = Path()
path.points = path_pts

print(path)

(0,0;300,0;300,127;0,127) w=0 bx=0 ex=0 r=false


In [8]:
#insert wg path points into wg pcell component
wg_imported = TOP.layout().create_cell("Waveguide", "GSiP", {"path":path, "radius":10, "width":0.5})
t = Trans(Trans.R0, 0, 0)
TOP.insert_cell(wg_imported, Point(0,0), 0)


In [9]:
layout.write("GSiP_loopback.gds")